In [1]:
from transformers import pipeline

d:\Anaconda\envs\llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


False

===================================BUG REPORT===================================
The following directories listed in your path were found to be non-existent: {WindowsPath('D')}
The following directories listed in your path were found to be non-existent: {WindowsPath('/172.28.1.128'), WindowsPath('http'), WindowsPath('8008')}
The following directories listed in your path were found to be non-existent: {WindowsPath('/172.28.1.128'), WindowsPath('http'), WindowsPath('8081')}
The following directories listed in your path were found to be non-existent: {WindowsPath('/172.28.1.128'), WindowsPath('http'), WindowsPath('8080')}
The following directories listed in your path were found to be non-existent: {WindowsPath('NFGwCwRrGk@mws02.mikr.us'), WindowsPath('40406/postgres'), WindowsPath('/postgres'), WindowsPath('postgresql')}
The following directories listed in your path were found to be non-existent: {WindowsPath('/172.28.1.129'), WindowsPath('http'), WindowsPath('8200')}
The following

d:\Anaconda\envs\llm\lib\site-packages\bitsandbytes\cuda_setup\main.py:166: UserWarning: Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes


  warn(msg)
d:\Anaconda\envs\llm\lib\site-packages\bitsandbytes\cuda_setup\main.py:166: UserWarning: D:\Anaconda\envs\llm did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)


RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):

        CUDA Setup failed despite GPU being available. Please run the following command to get more information:

        python -m bitsandbytes

        Inspect the output of the command and see if you can locate CUDA libraries. You might need to add them
        to your LD_LIBRARY_PATH. If you suspect a bug, please take the information from python -m bitsandbytes
        and open an issue at: https://github.com/TimDettmers/bitsandbytes/issues

In [ ]:
clf = pipeline("sentiment-analysis","distilbert-base-uncased-finetuned-sst-2-english")